<a href="https://colab.research.google.com/github/ttogle918/ds-section4-sprint2/blob/master/N422_distributed_representation/N422a_Distributed_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 2 / NOTE 2*

# 📝 Assignment

---


# Distributed_Representation

## Code

캐글의 [SMS Spam dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) 에 사전 학습된 Word2Vec 임베딩 벡터를 적용하여 분류해봅시다.<br/>
세션 노트에 있었던 단어 임베딩 벡터를 평균내어 분류하는 방법을 적용해봅시다.

In [9]:
!pip install gensim --upgrade

     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [10]:
import gensim

gensim.__version__

'4.1.2'

In [11]:
# 구글 뉴스 말뭉치로 학습된 word2vec 벡터 다운
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


### 1. Word2Vec과 코사인 유사도

word2vec을 이용해 구한 'data'와 'science'임베딩 값의 코사인 유사도를 구해보겠습니다. sklearn의 cosine_similarity를 이용하겠습니다.

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(wv['data'].reshape(1, -1), wv['science'].reshape(1, -1))

array([[0.1575913]], dtype=float32)

문항 1) 위에서 구한 코사인 유사도를 소수점 3째자리까지 입력하세요

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

### 2. 텍스트 분류

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다 `encoding = 'latin-1'` 을 사용합니다.
- 필요없는 열(column)을 삭제합니다.
- LabelEncoder를 사용하여 label 전처리를 해줍니다. 

In [ ]:
# from google.colab import files

# file = files.upload()

Saving spam.csv to spam.csv


In [18]:
# 로컬 파일 업로드
# google 드라이브에 로컬 파일 업로드
from google.colab import files, drive
drive.mount('/content/drive')


# 데이터 불러오기
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/code_states/spam.csv', encoding='latin-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [20]:
print(df.shape)
df.isna().sum()

(5572, 5)


v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [21]:
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [22]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [29]:
# LabelEncoder를 사용하여 label 전처리를 해줍니다.
encoder = LabelEncoder()
df.v1 = encoder.fit(df.v1).transform(df.v1)

### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 15%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words = 1000` 으로 설정합니다.
- pad_sequence의 `maxlen=150` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=64, epochs=10, validation_split=0.2` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [32]:
np.random.seed(42)
tf.random.set_seed(42)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(df.v2, df.v1, test_size=0.15, random_state=42)

In [41]:
# Tokenizer
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)

In [43]:
# pad_sequence
X_encoded = tokenizer.texts_to_sequences(X_train)

In [44]:
X_train=pad_sequences(X_encoded, maxlen=150, padding='post')
y_train=np.array(y_train)

In [45]:
# train
embedding_matrix=np.zeros((len(tokenizer.word_index)+1,300))

def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None

for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [50]:
# 학습 시, 파라미터는 batch_size=64, epochs=10, validation_split=0.2 로 설정합니다.
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=150, trainable=False))
model.add(GlobalAveragePooling1D()) # 입력되는 단어 벡터의 평균을 구하는 함수입니다.
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
60/60 [==============================] - 4s 14ms/step - loss: 0.6736 - acc: 0.8577 - val_loss: 0.6535 - val_acc: 0.8660
Epoch 2/10
60/60 [==============================] - 1s 9ms/step - loss: 0.6379 - acc: 0.8648 - val_loss: 0.6198 - val_acc: 0.8660
Epoch 3/10
60/60 [==============================] - 0s 8ms/step - loss: 0.6067 - acc: 0.8648 - val_loss: 0.5907 - val_acc: 0.8660
Epoch 4/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5800 - acc: 0.8648 - val_loss: 0.5652 - val_acc: 0.8660
Epoch 5/10
60/60 [==============================] - 1s 8ms/step - loss: 0.5569 - acc: 0.8648 - val_loss: 0.5436 - val_acc: 0.8660
Epoch 6/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5370 - acc: 0.8648 - val_loss: 0.5249 - val_acc: 0.8660
Epoch 7/10
60/60 [==============================] - 1s 10ms/step - loss: 0.5198 - acc: 0.8648 - val_loss: 0.5084 - val_acc: 0.8660
Epoch 8/10
60/60 [==============================] - 0s 6ms/step - loss: 0.5049 - acc: 0.

In [54]:
# evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]
X_test=tokenizer.texts_to_sequences(X_test)
X_test=pad_sequences(X_test, maxlen=150, padding='post')


model.evaluate(X_test, y_test)

27/27 [==============================] - 0s 5ms/step - loss: 0.4707 - acc: 0.8708


[0.4707179069519043, 0.8708133697509766]

### 3)Word2Vec에서의 OOV 문제

```
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
```
Lecture Note에 있는 위의 코드를 변형하여, OOV 개수를 확인해주세요.
- tokenizer는 위에서 활용한 tokenizer를 그대로 사용하겠습니다.
- Tip : dictionary를 활용하거나, Counter를 활용해보세요.

In [ ]:
### 이곳에서 과제를 수행해 주세요 ###
print(len(oov))